# Eco Rise Dashboard

## Libraries

In [1]:
# Libraries

# Data
import os # Operating system library
import numpy as np
import pandas as pd # Dataframe manipulations
import geopandas as gpd
from pathlib import Path # file paths

# Dash App
from jupyter_dash import JupyterDash # for running in a Jupyter Notebook
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output, State, ALL, MATCH

# Data Visualization
import plotly.express as px
import plotly.graph_objects as go

# Geojson loading
from urllib.request import urlopen
import json

## Data Cleaning and Piping

### Load Data

In [4]:
!ls ..

JupyterNotebooks      ecorise_dashboard.py  env
README.md             ecorise_datatables.py environment.yml
data                  ecorise_sunburst.py


In [179]:
# EcoRise Data
## Load data
programs = pd.read_csv('../data/Programs.csv')
organizations = pd.read_csv('../data/Organizations.csv')
geo = pd.read_excel('../data/Geodata.xlsx')

# Merge Organization data with geodata
orgs = organizations.merge(geo, left_on='Organization', right_on='ORGANIZATION', how='left') 

In [181]:
orgs[['Organization', 'Latitude', 'Longitude', 'LATITUDE', 'LONGITUDE']]

,Organization,Latitude,Longitude,LATITUDE,LONGITUDE
0,8 Shields Institute,29.724215,-95.405872,29.724215,-95.405872
1,Abilene Zoo,32.437144,-99.690826,32.437144,-99.690826
2,Alamo Area Master Naturalist,29.532410,-98.480730,29.532410,-98.480730
3,Aldine ISD,30.026326,-95.387047,30.026326,-95.387047
4,Alliance to Save Energy,38.905369,-77.042950,38.905369,-77.042950
...,...,...,...,...,...
272,Will Smith Zoo School,29.464280,-98.478210,29.464280,-98.478210
273,William R. Sinkin Eco Centro,29.446228,-98.493947,29.446228,-98.493947
274,Wyler Area Tramway State Park [TPWD],31.809387,-106.478149,31.809387,-106.478149
275,YWCA San Antonio,29.418425,-98.548717,29.418425,-98.548717


In [46]:
# Education Service Center Geospatial data layer - use geojson simplified to 0.01 threshold on QGIS 
## load education service ceters geojson
esc_simple_geojson = '../data/esc_simple.geojson'

with open(esc_simple_geojson) as json_file:
    esc_geojson = json.load(json_file)
    
# Load ESC Centroid data as point data (file generated using QGIS Centroid feature)
centroids = pd.read_csv('../data/centroids.csv')

### Add unique Identifiers to datasets

In [47]:
# df['unique_id'] = df.longstrings.map(hash)
orgs['orgID'] = orgs.index + 1
programs['programID'] = programs.index + 1

In [48]:
# Add OrgID column to Programs, merging on Organization
org_cols = ['Organization','orgID']
programs_org = orgs[org_cols]
programs = programs.merge(programs_org, how='left', on='Organization')
programs['orgID'] = programs['orgID'].astype('Int64')

##  brand colors

In [11]:
# Set Color scales for Figures using the EcoRise brand color palette
fulltint = ['#00A887','#B9D535','#FFC600','#FF8F12','#FF664B']
tint_75 = ['#40BEA5','#CBE068','#FFC400','#FFAB4D','#FF8C78']
tint_50 = ['#80D4C3','#DCEA9A','#FFE380','#FFC789','#FFB3A5']
eco_color = fulltint + tint_75 + tint_50
eco_color_r = fulltint[::-1] + tint_75[::-1] + tint_50[::-1]
eco_color_desc = eco_color[::-1]
eco_color_desc_r = eco_color_r[::-1]

### Lists
To DO: switch these to being dynamically driven from data source

In [7]:
Education_Service_Centers =['All Regions',
'Region 1 – Edinburg',
'Region 2 – Corpus Christi',
'Region 3 – Victoria',
'Region 4 – Houston',
'Region 5 – Beaumont',
'Region 6 – Huntsville',
'Region 7 – Kilgore',
'Region 8 – Mount Pleasant',
'Region 9 – Wichita Falls',
'Region 10 – Richardson',
'Region 11 – Fort Worth',
'Region 12 – Waco',
'Region 13 – Austin',
'Region 14 – Abilene',
'Region 15 – San Angelo',
'Region 17 – Lubbock',
'Region 18 – Midland',
'Region 19 – El Paso',
'Region 20 – San Antonio']

In [8]:
directory_org_cols = [
'Organization',
'City',
'Education_Service_Center',
'Full-Time_Staff',
'General_Email',
'Mission',
'Part-Time_Staff',
'Phone',
'Primary_Email',
'Primary_First_Name',
'Primary_Last_Name',
'Primary_Role',
'Custom_Region',
'Sector',
'Service_Area',
'Stakeholder_Category',
'State',
'Street_Address',
'Other_Staff/Contractors',
'Work_Terms',
'Website',
'ZIP_Code']

directory_program_cols =[
'Program',
'Organization',
'Title_I_School_Participants',
'Status',
'COVID-19_Adaptations',
'Description',
'Environmental_Themes',
'Regions_Served',
'Impact_Evaluation',
'Groups_Served',
'Program_Locations',
'Other_Languages',
'Services_&_Resources',
'Rural_Communities_Focus',
'Engaging_Rural_Communities',
'Schools_Served',
'Academic_Standards_Alignment',
'Student_Engagement_Frequency',
'Students_Served',
'Teacher/Administrator_Engagement_Frequency',
'Teachers/Administrators_Served',
'Program_Times',
'Title_I_Schools_&_Low_Socioeconomic_Background_Focus',
'Participants_Served',
'Academic_Standards'] 

### Roll up Environmental theme data

In [26]:
# List of standardized themes from dataset
# **Added to standardized list from spreadsheet: Conservation (Wildlife/Habitat), Green Building,  Outdoor Learning
themes_list = ['Outdoor learning',
'Waste',
'Water',
'Energy',
'Green building',
'Food Systems/Nutrition',
'Transportation',
'Air Quality',
'Conservation (wildlife/habitat)',
'Workforce Development',
'STEM']
add_themes = ['Conservation (Wildlife/Habitat)', 'Green Building',  'Outdoor Learning']
full_list = themes_list + add_themes 

def count_env_themes(programs, theme_list):
    themes = programs[['Program','Environmental_Themes']]
    expanded = themes['Environmental_Themes'].str.get_dummies(', ')
    # Replace columns that aren't in list with 'Other'
    expanded = expanded.rename(lambda x:  x if x in theme_list else 'Other', axis=1)
    # Group 'Other' Columns together
    expanded = expanded.groupby(expanded.columns, axis=1).sum()
    # Merge the themes and expanded dataframes
    themes = pd.concat([themes, expanded], axis=1)

    # Get Count of Programs per theme
    theme_count = pd.DataFrame(expanded.sum())
    theme_count.reset_index(inplace=True)
    theme_count.columns = ['Theme','Count']
    theme_count['Percent'] = round(100 * theme_count['Count'] / len(programs))
    theme_count = theme_count.astype({'Percent': int})
    theme_count = theme_count.sort_values(by=['Count'])
    print(theme_count)
    return theme_count

theme_count = count_env_themes(programs, full_list)

                              Theme  Count  Percent
5                             Other     19        6
8                    Transportation     26        8
4                    Green Building     47       15
0                       Air Quality     50       16
11            Workforce Development     52       16
2                            Energy     71       22
9                             Waste     82       26
3            Food Systems/Nutrition     85       27
10                            Water    174       55
7                              STEM    187       59
6                  Outdoor Learning    226       72
1   Conservation (Wildlife/Habitat)    234       74


In [178]:
# programs['Environmental_Themes']

df = programs.loc[:, ['Program','Environmental_Themes']]
df['Environmental_Themes'] = df['Environmental_Themes'].str.split(', ')
df = df.explode('Environmental_Themes')
# df['Environmental_Themes'].str.contains("Air Quality") 
df[df['Environmental_Themes'] == 'Outdoor Learning']
# df

,Program,Environmental_Themes
0,5th Grade Wetlands,Outdoor Learning
1,A Closer Look at Nature,Outdoor Learning
2,A Closer Look at Nature,Outdoor Learning
3,Activity Kits,Outdoor Learning
4,After School Club,Outdoor Learning
...,...,...
307,Wetlands Ecology Program,Outdoor Learning
308,WILD @ The Preserve,Outdoor Learning
312,"Youth Development, Stewardship & Outdoor Recre...",Outdoor Learning
313,Youth Programs,Outdoor Learning


In [147]:
d_filter = pd.read_csv('../data/filter_dict.csv')

## Function to produce dataframe with: term, display_term
def get_display_terms(filter_df, table, column):
    filter_df = filter_df[filter_df['table_name'] == table]
    filter_df = filter_df[filter_df['column_name'] == column]
    filter_df = filter_df.loc[:, ['term','display_term']]
    filter_df = filter_df.set_index('term')
    return filter_df

# get_env_theme_display_terms(d_filter)

## Function to produce dataframe with: Environmental_Themes, Count, Percent
def count_env_themes_for_programs(programs_df):
    ## Get program names and environmental themes
    program_themes_df = programs_df.loc[:, ['Program','Environmental_Themes']]
    ## Split environmental themes from string into an array of strings on ", "
    program_themes_df['Environmental_Themes'] = program_themes_df['Environmental_Themes'].str.split(', ')
    ## Explode dataframe by environmental theme to get each theme from within an array into a row
    program_themes_df = program_themes_df.explode('Environmental_Themes')
#     theme_list = program_themes_df['Environmental_Themes'].unique()
    ## Group by environmental theme counting the number of programs with each theme
    theme_count_df = program_themes_df.groupby(['Environmental_Themes']).count()
    ## Add column for percent of programs with the theme based on the number of program records
    theme_count_df['Percent'] = round(100 * theme_count_df['Program'] / len(programs_df))
    theme_count_df = theme_count_df.rename(columns={'Program': 'Count'})
    theme_count_df = theme_count_df.astype({'Percent': int})
#     print(theme_count_df)
    return theme_count_df

# count_env_themes_for_programs(programs)

def prepare_env_themes_for_graph(programs_df, filter_df):
    env_themes_df = count_env_themes_for_programs(programs_df)
    env_theme_terms = get_display_terms(filter_df, 'Programs', 'Environmental_Themes')
    ## Join environmental themes found in data with expected controlled terms and display terms
    env_themes_df = env_themes_df.join(env_theme_terms)
    ## Add column for label, using diplay term and percent value
    env_themes_df['Label'] = env_themes_df['display_term'] + ' ' + env_themes_df['Percent'].astype(str) + '%'
    ## Remove themes found within data that don't conform to recognized controlled terms (too bad, so sad)
    env_themes_df = env_themes_df[env_themes_df['Label'].notna()]
#     env_themes_df = env_themes_df[env_themes_df['Percent'] > 1]
    env_themes_df = env_themes_df.sort_values(by=['Count'])
    return env_themes_df

prepare_env_themes_for_graph(programs, d_filter)

,Count,Percent,display_term,Label
Environmental_Themes,,,,
Other,1,0,Other,Other 0%
Transportation,26,8,Transportation,Transportation 8%
Green Building,47,15,Green Building,Green Building 15%
Air Quality,50,16,Air Quality,Air Quality 16%
Workforce Development,52,16,Workforce Development,Workforce Development 16%
Energy,71,22,Energy,Energy 22%
Waste,82,26,Waste,Waste 26%
Food Systems/Nutrition,85,27,Food Systems,Food Systems 27%
Water,174,55,Water,Water 55%


In [209]:
df_o = orgs
def make_groupby_pie_chart(filter_df, df, col, textinfo = None, group = 'Organization', color_scale = eco_color,showlegend=False ):    
    pie_df = pd.DataFrame(df.groupby(col)[group].count())
#     print(pie_df)
    terms_df = get_display_terms(filter_df, 'Organizations', col)
#     print(terms_df)
    pie_df = pie_df.join(terms_df)
    print(pie_df)
    pie_df.reset_index(level=0, inplace=True)
    fig = px.pie(pie_df, values=group, names=pie_df['display_term'], color_discrete_sequence=color_scale)
    fig.update_traces(textposition='inside', textinfo=textinfo)
    fig.update_layout(showlegend=showlegend,height=250,   margin=dict(l=20, r=20, t=0, b=0)) 
    return fig

make_groupby_pie_chart(d_filter, df_o,'Sector',showlegend=True)

                                                 Organization  \
Sector                                                          
Business                                                   11   
Government (City, County, State, Federal, Etc.)            80   
Higher Education Institution                               17   
K-12 School/District                                       15   
Non-profit                                                153   
Other                                                       1   

                                                                 display_term  
Sector                                                                         
Business                                                             Business  
Government (City, County, State, Federal, Etc.)                    Government  
Higher Education Institution                     Higher Education Institution  
K-12 School/District                                     K-12 School/District  

In [105]:
# load filter dictionary data
d_filter = pd.read_csv('../data/filter_dict.csv')

# Create dictionary of filter options {tablename:{columnname:{'display_name':display_name,'options'{data_column:display_name}}}}
filter_dict = {}
for t in d_filter['table_name'].unique():
    t_df = d_filter[d_filter['table_name']==t][['column_name','display_name']].drop_duplicates()
    t_df = t_df.set_index('column_name')
    t_dict = t_df.to_dict('index')
    filter_dict[t] = t_dict
    for k in filter_dict[t].keys():
        tk_df = d_filter[(d_filter['table_name']==t) & (d_filter['column_name']==k)][['term','display_term']].drop_duplicates()
        tk_dict = tk_df.set_index('term').T.to_dict('records') 
        filter_dict[t][k]['options'] = tk_dict
print(filter_dict)

{'Organizations': {'Custom_Region': {'display_name': 'Region', 'options': [{'DFW/North TX': 'North Texas', 'Houston': 'Houston', 'El Paso': 'El Paso', 'San Antonio': 'San Antonio', 'Austin/Central TX': 'Central TX', 'RGV': 'Rio Grande Valley', 'Coastal Bend & Laredo': 'Coastal Bend & Laredo', 'Great Plains': 'Great Plains', 'Statewide & Other': 'Statewide & Other', 'National': 'National'}]}, 'Education_Service_Center': {'display_name': 'ESC Headquarters', 'options': [{'1': 'Region 1 – Edinburg', '2': 'Region 2 – Corpus Christi', '3': 'Region 3 – Victoria', '4': 'Region 4 – Houston', '5': 'Region 5 – Beaumont', '6': 'Region 6 – Huntsville', '7': 'Region 7 – Kilgore', '8': 'Region 8 – Mount Pleasant', '9': 'Region 9 – Wichita Falls', '10': 'Region 10 – Richardson', '11': 'Region 11 – Fort Worth', '12': 'Region 12 – Waco', '13': 'Region 13 – Austin', '14': 'Region 14 – Abilene', '15': 'Region 15 – San Angelo', '16': 'Region 16 - Amarillo', '17': 'Region 17 – Lubbock', '18': 'Region 18 – M

## Dash App Functions
### For figures

In [93]:
# Create scatter map figure
def make_scatter_map(filtered_df, lat = 'LATITUDE', lon = 'LONGITUDE', color = None, color_discrete_sequence = eco_color):
    fig = px.scatter_mapbox(filtered_df,
                            lat=filtered_df[lat],
                            lon=filtered_df[lon],
                            hover_name="Organization",
                            color = color,
#                             center = {'lat': 31.9686, 'lon': -99.9018},
                            zoom=4)
    fig.update_layout(mapbox_style="open-street-map")                  
    fig.update_geos(fitbounds="locations")
    fig.update_layout(
        showlegend=False,
        autosize=False,
        width=350,
        height=350,
        margin=dict(l=20, r=20, t=20, b=20))
    return fig

def make_groupby_pie_chart(df,col, textinfo = None, groupby_column = 'Organization', color_scale = eco_color,showlegend=False ):    
    df = pd.DataFrame(df.groupby(col)[groupby_column].count())
    df.reset_index(level=0, inplace=True)
    fig = px.pie(df, values=groupby_column, names=col, color_discrete_sequence=color_scale)
    fig.update_traces(textposition='inside', textinfo=textinfo)
    fig.update_layout(showlegend=showlegend,height=250,   margin=dict(l=20, r=20, t=0, b=0)) 
    return fig

def make_bar(df,xaxis,yaxis,label, orientation='h', textposition='inside', marker_color=eco_color_desc):
    fig = px.bar(df, x=xaxis, y=yaxis, orientation=orientation, text=label)
    fig.update_traces(marker_color=marker_color, texttemplate='%{text}', textposition=textposition)
#     fig.update_layout(yaxis_title = '')
    fig.update_yaxes(visible=False, showticklabels=False)
    fig.update_xaxes(visible=False, showticklabels=False)
    fig.update_layout(showlegend=False, margin=dict(l=20, r=20, t=0, b=0),
        height=250)   
    fig.update_layout(uniformtext_minsize=12)
    fig.update_traces(
       hovertemplate=None,
       hoverinfo='skip'
    )
    return fig

In [94]:
def make_map(orgdata, choro_geojson, featureidkey, choro_df, choro_df_location, choro_df_value, zoom = 5):
    # Design point layer
    scatter_fig_hover_template = '<b>%{hovertext}</b><br>Education Service Center: %{customdata[4]}<br><br>Region: %{customdata[0]}<br>Category: %{customdata[1]}<br>Sector: %{customdata[2]}<br>Service Area: %{customdata[3]}'
    scatter_fig = px.scatter_mapbox(orgdata, lat="LATITUDE", lon="LONGITUDE", 
                             hover_name="Organization", hover_data=["Custom_Region", 'Stakeholder_Category', 'Sector', 'Service_Area','Education_Service_Center'])
    scatter_fig.update_traces(hovertemplate=scatter_fig_hover_template)

    # Build choropleth layer
    fig = px.choropleth_mapbox(choro_df, geojson=choro_geojson, 
              featureidkey=featureidkey,
              locations=choro_df_location,
              color=choro_df_value,
              color_continuous_scale = tint_50,
              opacity = 0.25,
               zoom=zoom,
              center= {'lat': 31.9686, 'lon': -99.9018},
              mapbox_style='carto-positron')
    fig.update_traces(hovertemplate='ESC: %{location}<br>Organizations: %{z}')
    
    # add pt layer to map
    for item in range(0,len(scatter_fig.data)):
        fig.add_trace(scatter_fig.data[item])
    fig.update_layout(mapbox_style="open-street-map") # Ensure don't need token
    fig.update_layout(
        showlegend=False,
        autosize=True,
        height=350,
        margin=dict(l=20, r=20, t=20, b=0))

    return fig

###  Build App components Functions

In [95]:
def make_dropdown(i, options, placeholder, multi = True):
    ''' Create a dropdown taking id, option and placeholder values as inputs. '''
    
    # Handle either list or options as inputs
    if isinstance(options, dict):
        opts = [{'label': options[k], 'value': k}
                            for k in sorted(options)]
    else: 
        opts = [{'label': c, 'value': c}
                            for c in sorted(options)]    
    
    # Return actual dropdown component
    return dcc.Dropdown(
                id = f"{i}",
                options=opts,
                multi=multi,
                placeholder=placeholder,
                )



In [96]:
def build_directory_table(table_id, df, display_cols):
    ''' Function to create the structure and style elements of both the Organization and Programs tables'''
    # Checks to add: 
#     * input dataframe
#     * display_cols are in list of columsn in dataframe

    data_table = dash_table.DataTable(
                    id=table_id,
                    columns=[{"name": i, "id": i} for i in df[display_cols].columns],
                    data=df.to_dict('records'),
                    sort_action="native",
                    sort_mode="multi",
                    page_action="native",
                    page_current= 0,
                    page_size= 5,
                    css=[{'selector': '.row', 'rule': 'margin: 0; flex-wrap: nowrap'},
                        {'selector':'.export','rule':'position:absolute;left:0px;bottom:-35px'}],
                    fixed_columns={'headers': True, 'data': 1},
                    style_as_list_view=True,
                    style_cell={'padding': '5px',
                        'maxWidth': '300px',
                        'textAlign': 'left',
                        'height': 'auto',
                        'whiteSpace': 'normal'
                               },
                    style_header={
                        'backgroundColor': 'white',
                        'fontWeight': 'bold'
                    },
                    style_data={'padding-left':'15px'},
                    style_table={'minWidth': '100%', 'maxWidth': 'none !important','overflowX': 'auto'},
                    export_format="xlsx",
                    export_headers="display"
                                    )
    return data_table

### Styling

In [97]:
#STYLES
# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "18rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "20rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

### Construct Dash APP

In [98]:
# Build App
external_stylesheets = [dbc.themes.LITERA]
# For Jupyter notebook - this switches to app = dash.Dash format for running on server
app = JupyterDash(external_stylesheets=external_stylesheets)  # Jupyter Notebook
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets) # Dash APP server

app.config.suppress_callback_exceptions = True

# Get dictionaries to use for Org and Program filter lists. Specify list to pull, and then generate the dictionary
# for these filters from the filter_dict file
# Organizations
org_filter_list = ['Custom_Region', 'Education_Service_Center', 'Sector', 'Service_Area']
org_filter_dict = {k: filter_dict['Organizations'].get(k, None) for k in (org_filter_list)}

# Programs
pg_filter_list = ['Regions_Served','Environmental_Themes','Services_&_Resources','Academic_Standards_Alignment',
                 'Program_Locations','Program_Times','Rural_Communities_Focus','Groups_Served','Title_I_School_Participants']
pg_filter_dict = {k: filter_dict['Programs'].get(k, None) for k in (pg_filter_list)}

# Build components
overview_msg = html.Div([
    html.H5(id='overview_msg')
])

dds_orgs =  html.Div(
        [make_dropdown(f'dd-org-{k}', org_filter_dict[k]['options'][0], 
                       org_filter_dict[k]['display_name'])  for k in org_filter_dict
         ]
)

dds_programs =  html.Div(
        [make_dropdown(f'dd-pg-{k}', pg_filter_dict[k]['options'][0],pg_filter_dict[k]['display_name'])  
         for k in pg_filter_dict
         ]
)

dashboard = html.Div([
    dbc.Row([
        dbc.Col([
            dcc.Graph(id = 'map')
        ],md=6), 
        dbc.Col([
            dcc.Graph(id = 'treemap')
        ],md=6),   
    ],
    ),
    dbc.Row([ 
        dbc.Col([
            dcc.Graph(id='chart_theme')
        ],md=6),     
        dbc.Col([        
            dcc.Graph(id='chart_sector')
        ],md=6),         
    ])

])

# Handle Education services centers separate to handle All?
# dd_esc = make_dropdown('ESC', Education_Service_Centers, 'Education Service Centers') 

sidebar = html.Div(
    [
        html.H4('Filter on'),
        html.H5('Organization Data'),
        dds_orgs,
        html.H5('Program Data'),
        dds_programs,
        html.Div(id='div-overview_msg')
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div([
    html.Div(id="out-all-types"),
    html.Div([
        dcc.Tabs([            
            dcc.Tab([dashboard],label='Dashboard', id='tab-dashboard'),  
            dcc.Tab(id='tab-orgs'),
            dcc.Tab(id='tab-programs'),   
        ])
    ])
],
    id="page-content", style=CONTENT_STYLE)

## LAYOUT
app.layout = html.Div([
    dcc.Store(id='filtered_data', storage_type='session'),
    sidebar,
    content
])

## CALLBACKS
@app.callback(
    [Output('out-all-types','children')
         ,Output('tab-orgs','children'),Output('tab-orgs','label')
         ,Output('tab-programs','children') ,Output('tab-programs','label')
         ,Output('map', 'figure'),Output('chart_theme', 'figure')
         ,Output('treemap', 'figure'),Output('chart_sector', 'figure')
    ],
    [Input(f'dd-org-{dd}', "value") for dd in org_filter_list]+
    [Input(f'dd-pg-{dd}', "value") for dd in pg_filter_list ],
) 
def dd_values(*vals):
    # initial dataframes
    df_o = orgs
    df_p = programs
    
    #iterate through organization columns and filter data
    i = 0
    for v in vals[0:len(org_filter_list)]:        
        if(v):
            df_o = df_o[df_o[org_filter_list[i]].isin(v)]  
        i += 1

    # select only programs in the org list, then filter on program filters         
    df_p = programs[programs['orgID'].isin(df_o['orgID'])] 
    p = i - len(org_filter_list)
    p_count = 0
    p_msg = 'programs: '
    for v in vals[len(org_filter_list):]:        
        if(v):
            for i in v:
                p_msg = p_msg + i
            df_p = df_p[df_p[pg_filter_list[p]].isin(v)]
            p_count += 1
        p += 1                   
    
    # if any program filter is selected, filter orgs to this
    if p_count > 0:
        df_o = df_o[df_o['orgID'].isin(df_p['orgID'])]

    # calculate Org message
    org_count = len(df_o)
    pg_count = len(df_p)
    orgs_msg = 'Organization Records (' + str(org_count) + ')' 
    pg_msg  = 'Program Records (' + str(pg_count) + ')'
    
    # Build Directory tables
    orgs_tab = html.Div([
        build_directory_table('table-orgs', df_o, directory_org_cols)
        ],style={'width':'100%'})
    
    programs_tab = html.Div([
          build_directory_table('table-programs', df_p, directory_program_cols)
        ],style={'width':'100%'})    

    # Build Figures 
    # Calculate theme split
    theme_count = count_env_themes(df_p, full_list)
    theme_count['Label'] = theme_count['Theme'] + ' - ' + theme_count['Percent'].astype(str) + '%'
    
    # build map
    # Get Count of entities per esc
    esc_count = pd.DataFrame(df_o['Education_Service_Center'].value_counts())
    esc_count = esc_count.reset_index().rename(columns={"index": "ESC", "Education_Service_Center": "Organizations"})
    map_fig = make_map(df_o, esc_geojson, 'properties.FID', esc_count, 'ESC', 'Organizations', zoom=4)
    
    # build treemap
    pg_count = pd.DataFrame(df_p['orgID'].value_counts())
    pg_count = pg_count.reset_index().rename(columns={"index": "orgID", "orgID": "Program_Count"})
    pg_count = pg_count.merge(orgs[['Organization','orgID','Education_Service_Center','Sector','Service_Area']], how='inner', on='orgID')
    path1=['Education_Service_Center','Sector','Organization']
    sb = pg_count.dropna(subset=path)
    sb['Education_Service_Center'] = 'ESC: ' + sb['Education_Service_Center'].astype('int').astype('str')

    tree_fig = px.sunburst(sb, path=path1, values = 'Program_Count',maxdepth=2)

    # Test section
    test_msg = ''
    # return values
    return (test_msg, orgs_tab, orgs_msg, programs_tab, pg_msg, 
            map_fig, 
            make_bar(theme_count, 'Percent','Theme','Label'),  
            tree_fig,
            make_groupby_pie_chart(df_o,'Service_Area',showlegend=True)
           )

## TO DO
## Add themes once the data processing is worked out
## Output: Output('chart_theme', 'figure'),

# RUN app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/
